In [189]:
import pandas as pd
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             matthews_corrcoef, recall_score, f1_score,
                             confusion_matrix, precision_score, classification_report)
from sklearn.ensemble import StackingClassifier
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# 0) Setup

In [218]:
data_path = r'C:\Users\brand\OneDrive - University of New Orleans\Fall 2023\Machine Learning I\Project 1\iris.csv'
data = pd.read_csv(data_path)

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 1) Classifiers

In [187]:
# Running 10 Fold cross-validations for the listed classifiers
classifiers = {
    "ETC": ExtraTreeClassifier(),
    "Bagging": BaggingClassifier(),
    "DTC": DecisionTreeClassifier(),
    "LR": LogisticRegression(max_iter=1000),
    "SVC": SVC(),
    "kNN": KNeighborsClassifier()
}

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for name, classifier in classifiers.items():
    cv_score = cross_val_score(classifier, X, y, cv=kf, scoring='accuracy')
    print(f"{name}: {cv_score.mean():.4f} (+/- {cv_score.std():.4f})")

ETC: 0.9400 (+/- 0.0629)
Bagging: 0.9467 (+/- 0.0499)
DTC: 0.9467 (+/- 0.0581)
LR: 0.9533 (+/- 0.0670)
SVC: 0.9600 (+/- 0.0533)
kNN: 0.9533 (+/- 0.0670)


# 2) Class-wise Performance Metrics

In [40]:
# Had to create my own function since the specificity function was bugging out for each class
def specificity_per_class(cm, label_index):

    true_negative = np.delete(np.delete(cm, label_index, axis=0), label_index, axis=1).sum()
    false_positive = np.delete(cm, label_index, axis=0)[:, label_index].sum()
    
    return true_negative / (true_negative + false_positive)

In [220]:
kf = StratifiedKFold(n_splits=10, shuffle=True)

for name, classifier in classifiers.items():
    # Calculating cross-validated scores
    cv_score = cross_val_score(classifier, X, y, cv=kf, scoring='accuracy')
    print(f"\n{name}: {cv_score.mean():.4f} (+/- {cv_score.std():.4f})")

    # Getting cross-validated predictions
    y_pred = cross_val_predict(classifier, X, y, cv=kf)
    
    # Calculating metrics
    acc = accuracy_score(y, y_pred)
    bal_acc = balanced_accuracy_score(y, y_pred)
    mcc = matthews_corrcoef(y, y_pred)
    cm = confusion_matrix(y, y_pred)
    cr = classification_report(y, y_pred, target_names=y.unique(), output_dict=True)
    
    # Displaying metrics
    print(f"Accuracy: {acc:.4f}")
    print(f"Balanced Accuracy: {bal_acc:.4f}")
    print(f"Matthews Correlation Coefficient: {mcc:.4f}")
    
    print("Confusion Matrix:")
    print(cm)
    
    print("\nClass-wise metrics:")
    for label, metrics in cr.items():
        if label in y.unique():
            print(f"\nLabel: {label}")
            print(f"Sensitivity/Recall: {metrics['recall']:.4f}")
            print(f"F1-score: {metrics['f1-score']:.4f}")
    labels_indices = range(len(cm))  # assuming cm is square

    for label_index in labels_indices:
        spec = specificity_per_class(cm, label_index)
        print(f"Specificity for class {label_index}: {spec:.4f}")


ETC: 0.9267 (+/- 0.0467)
Accuracy: 0.9467
Balanced Accuracy: 0.9467
Matthews Correlation Coefficient: 0.9202
Confusion Matrix:
[[50  0  0]
 [ 0 45  5]
 [ 0  3 47]]

Class-wise metrics:

Label: Iris-setosa
Sensitivity/Recall: 1.0000
F1-score: 1.0000

Label: Iris-versicolor
Sensitivity/Recall: 0.9000
F1-score: 0.9184

Label: Iris-virginica
Sensitivity/Recall: 0.9400
F1-score: 0.9216
Specificity for class 0: 1.0000
Specificity for class 1: 0.9700
Specificity for class 2: 0.9500

Bagging: 0.9467 (+/- 0.0653)
Accuracy: 0.9533
Balanced Accuracy: 0.9533
Matthews Correlation Coefficient: 0.9301
Confusion Matrix:
[[50  0  0]
 [ 0 47  3]
 [ 0  4 46]]

Class-wise metrics:

Label: Iris-setosa
Sensitivity/Recall: 1.0000
F1-score: 1.0000

Label: Iris-versicolor
Sensitivity/Recall: 0.9400
F1-score: 0.9307

Label: Iris-virginica
Sensitivity/Recall: 0.9200
F1-score: 0.9293
Specificity for class 0: 1.0000
Specificity for class 1: 0.9600
Specificity for class 2: 0.9700

DTC: 0.9400 (+/- 0.0757)
Accuracy

# 3) Stacking

In [10]:
# Metrics evaluation function
def evaluate_metrics(y_true, y_pred):
    # Compute Sensitivity and Specificity for multiclass
    cm = confusion_matrix(y_true, y_pred)
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    
    # Sensitivity
    TPR = TP / (TP + FN)
    sensitivity = np.mean(TPR)
    
    # Specificity
    TNR = TN / (TN + FP)
    specificity = np.mean(TNR)
    
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "Matthews Correlation Coefficient": matthews_corrcoef(y_true, y_pred),
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "F1-Score": f1_score(y_true, y_pred, average='macro'),
        "Confusion Matrix": cm
    }
    
    return metrics

In [219]:
# Base classifiers and meta classifiers for the stacking models
base_classifiers_1 = [('ETC', ExtraTreeClassifier()),
                      ('Bagging', BaggingClassifier()),
                      ('DTC', DecisionTreeClassifier())]

base_classifiers_2 = [('LR', LogisticRegression(max_iter=1000)),
                      ('SVC', SVC(probability=True)),
                      ('kNN', KNeighborsClassifier())]

meta_classifier_1 = LogisticRegression(max_iter=1000)
meta_classifier_2 = DecisionTreeClassifier()

# Construct the two stacked classifiers
stacking_clf_1 = StackingClassifier(estimators=base_classifiers_1, final_estimator=meta_classifier_1, stack_method='predict_proba')
stacking_clf_2 = StackingClassifier(estimators=base_classifiers_2, final_estimator=meta_classifier_2, stack_method='predict_proba')

# Training both
for clf in [stacking_clf_1, stacking_clf_2]:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    metrics = evaluate_metrics(y_test, y_pred)
    for metric_name, metric_value in metrics.items():
        if metric_name != "Confusion Matrix":
            print(f"{metric_name}: {metric_value:.4f}")
        else:
            print(f"{metric_name}:\n{metric_value}")
    print("-" * 50)

Accuracy: 0.9333
Balanced Accuracy: 0.9444
Matthews Correlation Coefficient: 0.9055
Sensitivity: 0.9444
Specificity: 0.9710
F1-Score: 0.9280
Confusion Matrix:
[[11  0  0]
 [ 0 10  2]
 [ 0  0  7]]
--------------------------------------------------
Accuracy: 0.9667
Balanced Accuracy: 0.9722
Matthews Correlation Coefficient: 0.9509
Sensitivity: 0.9722
Specificity: 0.9855
F1-Score: 0.9633
Confusion Matrix:
[[11  0  0]
 [ 0 11  1]
 [ 0  0  7]]
--------------------------------------------------
